In [94]:
# %load_ext autoreload
# %autoreload 1
# %aimport foo
import os
import sys
import math
from packages.models import Model
from packages.helper import traj_speed, play_trajs, beta, min_dist, scatter3d
import numpy as np
from scipy.stats import ttest_rel, ttest_ind
from numpy import gradient
from numpy.linalg import norm
import pickle
import matplotlib.pyplot as plt
from packages.tester import Tester
from packages.simulator import Agent
from packages import data_container
from mpl_toolkits.mplot3d import Axes3D
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container
%matplotlib qt


# Steering to goal

In [109]:
file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Fajen_steer_exp1a_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

In [135]:
set(data.info['trial_id'])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79}

In [116]:
'''Animate data'''
%matplotlib qt
i = 51
# p_obst = np.array(data.info['p_obst'][i])
t0 = data.info['goal_onset'][i]
p_goal = np.array(data.info['p_goal'][i])
p_subj = np.array(data.get_traj(i))[:,:2]
trajs = [p_goal, p_subj]
ws = [data.info['w_goal'], 0.4]
play_trajs(trajs, ws, data.Hz)

In [113]:
'''Plot speed'''
%matplotlib qt
Hz = data.Hz
plt.figure()
for i in range(len(data.trajs)):
    subj = np.array(data.get_traj(i))[:, :2]
    if data.info['subj_id'][i] == 5 and abs(data.info['goal_angle'][i]) == 25:
        for j in range(1, len(subj)):
            spd1 = norm([(a - b) * Hz for a, b in zip(subj[j], subj[j-1])])
            spd2 = norm([(a - b) * Hz for a, b in zip(subj[j+1], subj[j])])
            if spd1 > 0.22 and spd2 > spd1:
                t0 = j
                break
        t1 = t0 + 4 * Hz
        spd = traj_speed(subj[:], data.Hz)
        plt.plot(spd)

In [136]:
'''Find preferred speed'''
spds = []
Hz = data.Hz
plt.figure()
for i in range(len(data.trajs)):
    subj = np.array(data.get_traj(i))[:, :2]
    spd = traj_speed(subj, Hz)[-2*Hz: -1*Hz]
    plt.plot(spd)
    spds.append(np.mean(spd))
print(np.mean(spds), np.std(spds), len(spds))

1.080102096073387 0.09249857229889018 640


In [142]:
'''Speed at stimuli onset'''
spds = np.zeros(80)
counts = np.zeros(80)
Hz = data.Hz
plt.figure()
for i in range(len(data.trajs)):
    subj = np.array(data.get_traj(i))[:, :2]
    spd = traj_speed(subj, Hz)
    onset = data.info['goal_onset'][i]
    i_trial = data.info['trial_id'][i]
    spds[i_trial] += spd[onset]
    counts[i_trial] += 1
spds /= counts
plt.plot(spds)
sum(counts)

640.0

In [4]:
'''Steer2Goal test'''

tests = []

w = 0.4
mass_spring_approach1 = {'name': 'mass_spring_approach1', 
                         'b1': 3.25, 'b2': 4.8, 'k1':7.5, 'k2': 6, 'c1': 0.4, 'c2': 0.4}

models = {'approach': mass_spring_approach1}
agent = Agent(1, goal_id=0, w=w, models=models)
info = {'w': {'goals': [0.1]}, 'ps':[]}
goals = []
subjs = []
obsts = []
Hz = data.Hz
for i in range(len(data.trajs)):
    if i in data.dump:
        continue

    goal = data.info['p_goal'][i]
    subj = np.array(data.get_traj(i))[:,0:2].tolist()
    # Define the start and end of a trial for fitting
#     t0 = data.info['goal_onset'][i]
    for j in range(1, len(subj)):
        spd1 = norm([(a - b) * Hz for a, b in zip(subj[j], subj[j-1])])
        spd2 = norm([(a - b) * Hz for a, b in zip(subj[j+1], subj[j])])
        if spd1 > 0.22 and spd2 > spd1:
            t0 = j
            break
    t1 = t0 + 4 * Hz
    goals.append([goal[t0:t1]])
    subjs.append(subj[t0:t1])
    info['ps'].append(data.info['ps'][i])


tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
tests[-1].simulate()

#===========================================


In [98]:
'''Test metrics'''
i_test = 0
s_RMSE = tests[i_test].test('s_RMSE')
print(np.mean(tests[i_test].test('p_dist')),
      np.mean(s_RMSE),
      np.mean(tests[i_test].test('phi_RMSE')),
      np.mean(tests[i_test].test('s_MAE')),
      np.mean(tests[i_test].test('phi_MAE')))

0.1793757953798874 0.036822098924205786 0.13074688665996761 0.028640737825079275 0.1107502526832798


In [6]:
'''Animate trial'''
%matplotlib qt
i_test = 0
i = 0
goal = tests[i_test].goals[i][0]
# obst = tests[i_test].obsts[i][0]
subjs = tests[i_test].subjs[i]
pred = tests[i_test].p_pred[i]
play_trajs([goal, subjs, pred], [None, 0.4, 0.4], tests[i_test].Hz, labels=('goal','subj','model'))
print(data.info['goal_angle'][i])
print(tests[i_test].test('p_dist', i_trial=i),       
      tests[i_test].test('s_RMSE', i_trial=i),
      tests[i_test].test('phi_RMSE', i_trial=i),
      tests[i_test].test('s_MAE', i_trial=i),
      tests[i_test].test('phi_MAE', i_trial=i))


-5
[0.06116874407789541] [0.04938083095492003] [0.014906020060634765] [0.04312975765040974] [0.01226704233326539]


In [48]:
'''Plot data and model'''
%matplotlib qt
i_test = 0
plt.figure()
for i in range(len(tests[i_test].subjs)):
    subjs = np.array(tests[i_test].subjs[i])
    pred = np.array(tests[i_test].p_pred[i])
    plt.plot(subjs[:,0], subjs[:,1], 'k')
    
plt.figure()
for i in range(len(tests[i_test].subjs)):
    subjs = np.array(tests[i_test].subjs[i])
    pred = np.array(tests[i_test].p_pred[i])
    plt.plot(pred[1:-1,0], pred[1:-1,1], 'b')

In [31]:
'''Find worst trials'''
i = np.argmax(s_RMSE)
# print(data.info['subj_id'][i], i, s_RMSE[i])
for i in range(len(s_RMSE)):
    if 0.1475 - s_RMSE[i] < 0.05:
        print(data.info['subj_id'][i], i, s_RMSE[i])

0 42 0.15013027324969733
1 99 0.13910930168959046
1 113 0.10631956769865859
2 221 0.11960961334537777
7 580 0.11195528213490742
7 601 0.11561771589717455


In [2]:
'''Loading fitting results'''
b2 = []
k2 = []
t_relax = []
s_RMSE = []
s_MAE = []
s_MSE = []

with open('fitting_results_Fajen_steer_exp1a_2020-6-9-19-8-16.csv', 'r') as file:
    for line in file:
        if line != '\n':
            line = line.split(',')
            args = line[2].split('/')
#             if float(args[1]) < 5 and float(args[3]) < 5:
            b2.append(float(args[1]))
            k2.append(float(args[3]))
            t_relax.append(float(args[-1]))
            err = line[4].split('/')        
            s_RMSE.append(float(err[0]))
            s_MAE.append(float(err[1]))
            s_MSE.append(float(err[2]))

In [3]:
'''Plot fitting results'''
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter3d(b2, k2, s_RMSE, fig=fig, ax=ax)
ax.set_xlabel('b2')
ax.set_ylabel('k2')
ax.set_zlabel('RMSE_s')

Text(0.5, 0, 'RMSE_s')

In [109]:
'''Find the best fit parameters'''
'''mass_spring_approach: t_relax = 0.7'''
'''mass_spring_approach1: b2 = 4.8, k2 = 6'''
i0 = np.argmin(s_RMSE)
print(t_relax[i0], s_RMSE[i0])
# print(b2[i0], k2[i0], s_RMSE[i0])
for i in range(len(b2)):
    if s_RMSE[i] - s_RMSE[i0] < 0.0001:
        print(b2[i], k2[i], s_RMSE[i])
        plt.scatter(k2[i], b2[i])
plt.plot([3.07, 9.98], [2.84, 7.58])

0.7027027027027026 0.0437


# movObst

In [153]:
file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_exp1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

In [155]:
set(data.info['trial_id'])

In [53]:
'''Animate data'''
%matplotlib qt
i = 630
print(data.info['subj_id'][i])
p_obst = np.array(data.info['p_obst'][i])
p_goal = np.array(data.info['p_goal'][i])
p_subj = np.array(data.get_traj(i))[:,:2]
trajs = [p_goal, p_obst, p_subj]
ws = [data.info['w_goal'], data.info['w_obst'], 0.4]
play_trajs(trajs, ws, data.Hz)

3


In [156]:
'''Speed at stimuli onset'''
spds = np.zeros(72)
counts = np.zeros(72)
Hz = data.Hz
plt.figure()
for i in range(len(data.trajs)):
    if i in data.dump: continue
#     if data.info['subj_id'][i] != 1: continue
    subj = np.array(data.get_traj(i))[:, :2]
    spd = traj_speed(subj, Hz)
    onset = data.info['stimuli_onset'][i]
    i_trial = data.info['trial_id'][i] - 1
    spds[i_trial] += spd[onset]
    counts[i_trial] += 1
spds /= counts
plt.plot(spds)
sum(counts)

1005.0

In [3]:
'''Plot speed by passing order'''
%matplotlib qt
# passing in front
# plt.figure()
for i in range(len(data.trajs)):
    if i not in data.dump and i==0:
        subj = np.array(data.get_traj(i))[:,:2]
        obst = np.array(data.info['p_obst'][i])
        angle = data.info['obst_angle'][i]
        t0 = data.info['stimuli_onset'][i]
        inx, min_d = min_dist(subj, obst)
        spd = traj_speed(subj[t0:], data.Hz)
        if min_d > 0: 
            plt.figure()
            plt.hist(spd, bins=np.linspace(0.025, 2.025, 41))
            plt.figure()
            plt.plot(spd)
# passing behind
# plt.figure()
for i in range(len(data.trajs)):
    if i not in data.dump and i==0:
        subj = np.array(data.get_traj(i))[:,:2]
        obst = np.array(data.info['p_obst'][i])
        angle = data.info['obst_angle'][i]
        t0 = data.info['stimuli_onset'][i]
        inx, min_d = min_dist(subj, obst)
        spd = traj_speed(subj[t0:], data.Hz)
        if min_d < 0:            
            plt.figure()
            plt.hist(spd, bins=np.linspace(0.025, 2.025, 41))
            plt.figure()
            plt.plot(spd)

In [86]:
'''Mean speed by timestamps'''
spd0 = [] # Speeds at the onset of the stimuli
spd1 = [] # Speeds at dca
spd2 = [] # Speeds at 1 second after dca
spd3 = [] # Speeds at 2 second after dca
for i in range(len(data.trajs)):
    if i in data.dump: continue
#     if i in [453, 597, 630]: continue
    if i in [563, 568, 569, 571, 575]: continue
    subj = np.array(data.get_traj(i))[:,:2]
    obst = np.array(data.info['p_obst'][i])
    inx, min_d = min_dist(subj, obst)
    if min_d > 0:
        t0 = data.info['stimuli_onset'][i]
        t1 = inx
        t2 = t1 + 1 * data.Hz
        t3 = t2 + int(0.5 * data.Hz)
        spd = traj_speed(subj, data.Hz)
        spd0.append(spd[t0])
        spd1.append(spd[t1])
        spd2.append(spd[t2])
        spd3.append(spd[t3])

print(np.mean(spd0), np.std(spd0), len(spd0))
print(np.mean(spd1), np.std(spd1), len(spd1))
print(np.mean(spd2), np.std(spd2), len(spd2))
print(np.mean(spd3), np.std(spd3), len(spd3))

1.057098158385917 0.14207330089091316 264
1.2134584143975224 0.1267496850798508 264
1.140598218046392 0.10999840554418058 264
1.0527282964140146 0.12099365311718355 264


In [93]:
i

1.140598218046392

In [91]:
'''t test'''
ttest_ind(e, f)

Ttest_indResult(statistic=-4.616279574336179, pvalue=4.4909622037383706e-06)

In [20]:
'''movObst test'''

tests = []

w = 0.4
mass_spring_approach = {'name': 'mass_spring_approach', 'ps': 1.3,
                        't_relax': 0.7,
                        'b': 3.25, 'k_g': 7.5, 'c1': 0.4, 'c2': 0.4}
mass_spring_approach1 = {'name': 'mass_spring_approach1', 
                         'b1': 3.25, 'b2': 4.8, 'k1':7.5, 'k2': 6, 'c1': 0.4, 'c2': 0.4}
cohen_avoid = {'name': 'cohen_avoid', 'ps': 1.3,
               'b1': 2.62, 'k1': 375.01, 'c5': 11.05, 'c6': 0.87,
               'b2': -0.68, 'k2': -476.03, 'c7': 57.84, 'c8': 18.18}
cohen_avoid3 = {'name': 'cohen_avoid3', 'k1': 530, 'k2': 500, 'c5': 6, 'c6': 1.3, 'c7': 6, 'c8': 1.3}
cohen_avoid4 = {'name': 'cohen_avoid4', 'k1': 50, 'k2': 50, 'c5': 20, 'c6': 1.3, 'c7': 100, 'c8': 1.3}

models = {'approach': mass_spring_approach, 'avoid': cohen_avoid}
agent = Agent(1, goal_id=0, w=w, models=models)
info = {'w': {'goals': [0.1], 'obsts': [0.1]}, 'ps': []}
goals = []
obsts = []
subjs = []
for i in range(len(data.trajs)):
    if i in data.dump:
        continue

    goal = data.info['p_goal'][i]
    obst = data.info['p_obst'][i]
    subj = np.array(data.get_traj(i))[:,0:2].tolist()
    # Define the start and end of a trial for fitting
    t0 = data.info['stimuli_onset'][i]
    for j in range(t0, len(subj)):
        p0 = subj[j]
        p1 = obst[j]
        v0 = [(a - b) * data.Hz for a, b in zip(subj[j+1], subj[j])]
        if abs(beta(p0, p1, v0)) > np.pi / 2:
            t1 = j
            break    
    goals.append([goal[t0:t1]])
    obsts.append([obst[t0:t1]])
    subjs.append(subj[t0:t1])
    info['ps'].append(data.info['ps'][i])

Hz = data.Hz
tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
tests[-1].simulate()

#===========================================

# models = {'approach': mass_spring_approach1, 'avoid': cohen_avoid4}
# agent = Agent(1, goal_id=0, w=w, models=models)
# tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
# tests[-1].simulate()

In [21]:
'''Test metrics'''
i_test = 0
s_RMSE = tests[i_test].test('s_RMSE')
print(np.mean(tests[i_test].test('p_dist')),       
      np.mean(s_RMSE),
      np.mean(tests[i_test].test('phi_RMSE')),
      np.mean(tests[i_test].test('s_MAE')),
      np.mean(tests[i_test].test('phi_MAE')))

0.19476057624702672 0.11680921773185737 0.1352756995510318 0.10011257551258704 0.10785796250919158


In [11]:
'''Animate trial'''
%matplotlib qt
# back 1,3,7 front 10,17,18, 117
i_test = 0
i = 3
goal = tests[i_test].goals[i][0]
obst = tests[i_test].obsts[i][0]
subjs = tests[i_test].subjs[i]
pred = tests[i_test].p_pred[i]
play_trajs([goal, obst, subjs, pred], [None, 0.1, 0.4, 0.4], tests[i_test].Hz, labels=('goal', 'obst', 'subj', 'model'))
print(tests[i_test].test('p_dist', i_trial=i),       
      tests[i_test].test('s_RMSE', i_trial=i),
      tests[i_test].test('phi_RMSE', i_trial=i),
      tests[i_test].test('s_MAE', i_trial=i),
      tests[i_test].test('phi_MAE', i_trial=i))

[0.10962716828645608] [0.07945198406905166] [0.13914797590369155] [0.07072183174237258] [0.09959991814636837]


# Cohen_avoid

In [13]:
'''Loading fitting results'''
b1 = []
b2 = []
k1 = []
k2 = []
c5 = []
c6 = []
c7 = []
c8 = []
p_dist = []
s_RMSE = []
phi_RMSE = []

with open('fitting_results_Cohen_movObst_exp1_2020-6-16-18-29-24.csv', 'r') as file:
    for line in file:
        if line != '\n':
            line = line.split(',')
            args = line[2].split('/')
            err = line[4].split('/') 
            if not np.isnan(float(err[0])):
                b1.append(float(args[5]))
                k1.append(float(args[6]))
                c5.append(float(args[7]))
                c6.append(float(args[8]))
                b2.append(float(args[9]))
                k2.append(float(args[10]))
                c7.append(float(args[11]))
                c8.append(float(args[12]))
                p_dist.append(float(err[0]))
                s_RMSE.append(float(err[1]))
                phi_RMSE.append(float(err[2]))

In [14]:
'''Plot fitting results'''
%matplotlib qt
x = []
y = []
z = []
for i in range(len(p_dist)):
    if b1[i] == 4 and k1[i] == 400 and c5[i] ==4 and c6[i] == 4 and \
       b2[i] > 0 and k2[i] > 0 and c7[i] == 6 and c8[i] == 2:
        x.append(b2[i])
        y.append(k2[i])
        z.append(s_RMSE[i])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter3d(x, y, z, fig=fig, ax=ax)
ax.set_xlabel('b2')
ax.set_ylabel('k2')
ax.set_zlabel('s_RMSE')

Text(0.5, 0, 's_RMSE')

In [107]:
%matplotlib qt
# plt.hist(p_dist,100)
set(c8)

{1.0, 2.0, 3.0, 4.0}

In [114]:
'''Find the best fit parameters'''
'''Cohen_avoid: b1 > 4, k1 < 400, c5 > 4, c6 > 4, 
                b2 < 1, k2 300, c7 < 6, c8 2, min(p_dist) = 0.2199'''
%matplotlib qt
i0 = np.argmin(p_dist)
err = []
print(b1[i0], k1[i0], c5[i0], c6[i0], b2[i0], k2[i0], c7[i0], c8[i0], p_dist[i0], s_RMSE[i0], phi_RMSE[i0],
      min(s_RMSE), min(phi_RMSE))
for i in range(len(p_dist)):
    if p_dist[i] - p_dist[i0] < 0.0006:
        print(b1[i0], k1[i], c5[i], c6[i], b2[i0], k2[i], c7[i], c8[i], p_dist[i])
    if c8[i] == 3:
        err.append(s_RMSE[i])
x = np.random.normal(size=len(err))
plt.scatter(x, err, s=1)
err.sort()
print(np.mean(err[:100]))

2.0 600.0 4.0 2.0 1.0 100.0 6.0 2.0 0.2199 0.1214 0.1765 0.1171 0.1555
2.0 600.0 4.0 2.0 1.0 100.0 6.0 2.0 0.22
2.0 533.33 4.0 2.0 1.0 100.0 6.0 2.0 0.2205
2.0 600.0 4.0 2.0 1.0 100.0 7.0 2.0 0.2201
2.0 533.33 4.0 2.0 1.0 100.0 7.0 2.0 0.2205
2.0 600.0 4.0 2.0 1.0 100.0 8.0 2.0 0.2202
2.0 600.0 4.0 2.0 1.0 100.0 9.0 2.0 0.2203
2.0 600.0 4.0 2.0 1.0 200.0 6.0 2.0 0.2203
2.0 600.0 4.0 2.0 1.0 200.0 6.0 3.0 0.2205
2.0 600.0 4.0 2.0 1.0 200.0 7.0 2.0 0.2201
2.0 533.33 4.0 2.0 1.0 200.0 7.0 2.0 0.2205
2.0 600.0 4.0 2.0 1.0 200.0 8.0 2.0 0.22
2.0 533.33 4.0 2.0 1.0 200.0 8.0 2.0 0.2204
2.0 600.0 4.0 2.0 1.0 200.0 9.0 2.0 0.2201
2.0 533.33 4.0 2.0 1.0 200.0 9.0 2.0 0.2204
2.0 600.0 4.0 2.0 1.0 300.0 6.0 3.0 0.2203
2.0 600.0 4.0 2.0 1.0 300.0 7.0 3.0 0.2204
2.0 600.0 4.0 2.0 1.0 300.0 8.0 2.0 0.2203
2.0 600.0 4.0 2.0 1.0 300.0 8.0 3.0 0.2205
2.0 600.0 4.0 2.0 1.0 300.0 9.0 2.0 0.2201
2.0 600.0 4.0 2.0 1.0 400.0 6.0 3.0 0.2201
2.0 533.33 4.0 2.0 1.0 400.0 6.0 3.0 0.2205
2.0 600.0 4.0 2.0 1.0 40

# Cohen_avoid2

# Cohen_avoid3 and 4

In [137]:
'''Loading fitting results'''
k1 = []
k2 = []
c5 = []
c6 = []
c7 = []
c8 = []
p_dist = []
s_RMSE = []
phi_RMSE = []

with open('fitting_results_Cohen_movObst_exp1_2020-6-11-13-17-55.csv', 'r') as file:
    for line in file:
        if line != '\n':
            line = line.split(',')
            args = line[2].split('/')
            err = line[4].split('/') 
            if not np.isnan(float(err[0])):
                k1.append(float(args[6]))
                k2.append(float(args[7]))
                c5.append(float(args[8]))
                c6.append(float(args[9]))
                c7.append(float(args[10]))
                c8.append(float(args[11]))
                p_dist.append(float(err[0]))
                s_RMSE.append(float(err[1]))
                phi_RMSE.append(float(err[2]))

In [ ]:
'''Plot fitting results'''
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(b2, k2, s_RMSE, marker='o', c='r')
ax.set_xlabel('b2')
ax.set_ylabel('k2')
ax.set_zlabel('s')

In [142]:
print(set(k1))
print(set(c5))
print(set(c6))
print(set(k2))
print(set(c7))
print(set(c8))

{100.0, 325.0, 175.0, 400.0, 250.0}
{0.5, 2.88, 5.25, 7.62, 10.0}
{0.5, 2.88, 5.25, 7.62, 10.0}
{100.0, 325.0, 175.0, 400.0, 250.0}
{0.5, 2.88, 5.25, 7.62, 10.0}
{0.5, 2.88, 5.25, 7.62, 10.0}


In [156]:
'''Find the best fit parameters'''
'''Cohen_avoid3: k1 around 200, c5 >= 7.5, c6 around 5, k2 >= 300, c7 around 5, c8 >= 7.5, min(p_dist) = 0.1897'''
'''Cohen_avoid4: b1 '''
%matplotlib qt
i0 = np.argmin(p_dist)
err = []
print(k1[i0], c5[i0], c6[i0], k2[i0], c7[i0], c8[i0], p_dist[i0], s_RMSE[i0], phi_RMSE[i0],
      min(s_RMSE), min(phi_RMSE))
for i in range(len(p_dist)):
    if p_dist[i] - p_dist[i0] < 0.0005:
        print(k1[i], c5[i], c6[i], k2[i], c7[i], c8[i], p_dist[i], s_RMSE[i], phi_RMSE[i])
    if c5[i] == 0.5:
        err.append(p_dist[i])
x = np.random.normal(size=len(err))
plt.scatter(x, err, s=1)
err.sort()
print(np.mean(err[:300]))


250.0 10.0 5.25 100.0 10.0 5.25 0.1897 0.1102 0.1368 0.104 0.133
100.0 7.62 10.0 100.0 10.0 5.25 0.1901 0.1102 0.1373
100.0 7.62 10.0 175.0 10.0 2.88 0.1901 0.1102 0.1374
175.0 10.0 7.62 100.0 7.62 2.88 0.1901 0.1099 0.1372
175.0 10.0 7.62 100.0 7.62 5.25 0.1901 0.1107 0.1366
175.0 10.0 7.62 100.0 10.0 5.25 0.1898 0.1102 0.1368
175.0 10.0 7.62 100.0 10.0 7.62 0.19 0.111 0.1366
175.0 10.0 7.62 175.0 7.62 2.88 0.1901 0.1108 0.1366
175.0 10.0 7.62 175.0 10.0 2.88 0.1898 0.1102 0.1368
175.0 10.0 7.62 250.0 10.0 2.88 0.19 0.1111 0.1366
250.0 10.0 5.25 100.0 7.62 2.88 0.19 0.1099 0.1373
250.0 10.0 5.25 100.0 7.62 5.25 0.1901 0.1107 0.1366
250.0 10.0 5.25 100.0 10.0 5.25 0.1897 0.1102 0.1368
250.0 10.0 5.25 100.0 10.0 7.62 0.19 0.111 0.1366
250.0 10.0 5.25 175.0 7.62 2.88 0.19 0.1107 0.1366
250.0 10.0 5.25 175.0 10.0 2.88 0.1897 0.1102 0.1368
250.0 10.0 5.25 250.0 10.0 2.88 0.19 0.1111 0.1366
325.0 7.62 2.88 100.0 10.0 5.25 0.1901 0.1101 0.1386
325.0 7.62 2.88 175.0 10.0 2.88 0.1901 0.1101 0.